# Particle Filters

* Easist to program and the most flexible among our three types of filters

![](images/13.1.png)

* Histogram: 
    * Discrete since dist defined over a finite set of bins.
        * Exponential (think of a grid in k dims)
* Kalman: Single Gaussian -> unimodal
    * Quadratic since only represented by mean and covariance matrix
    * Approximate since KFs are only exact for linear systems

* Particle filters:
    * Scale exponentially in some applications (don't use for more than 4 dim)
    * Scale well in some trackiing applications


![](images/13.2.png)
![](images/13.3a.png)
![](images/13.3b.png)
![](images/13.3c.png)

* Blue stripes: Sonar sensors (sound)
* Each red dot is a discrete guess as to where we might be (x, y coord, heading)

* Particles that are more consistent with the measurement are more likely to survive

* Two clouds until you enter one of the offices because because the corridor is symmetric

### Code
#### Robot Class code

![](images/13.4.png)

### Robot class

In [6]:
from math import *
import random

In [7]:
landmarks  = [[20.0, 20.0], [80.0, 80.0], [20.0, 80.0], [80.0, 20.0]]
world_size = 100.0

In [8]:
class robot:
    def __init__(self):
        self.x = random.random() * world_size
        self.y = random.random() * world_size
        self.orientation = random.random() * 2.0 * pi
        self.forward_noise = 0.0;
        self.turn_noise    = 0.0;
        self.sense_noise   = 0.0;
    
    def set(self, new_x, new_y, new_orientation):
        if new_x < 0 or new_x >= world_size:
            raise ValueError('X coordinate out of bound')
        if new_y < 0 or new_y >= world_size:
            raise ValueError('Y coordinate out of bound')
        if new_orientation < 0 or new_orientation >= 2 * pi:
            raise ValueError('Orientation must be in [0..2pi]')
        self.x = float(new_x)
        self.y = float(new_y)
        self.orientation = float(new_orientation)
    
    
    def set_noise(self, new_f_noise, new_t_noise, new_s_noise):
        # makes it possible to change the noise parameters
        # this is often useful in particle filters
        self.forward_noise = float(new_f_noise);
        self.turn_noise    = float(new_t_noise);
        self.sense_noise   = float(new_s_noise);
    
    
    def sense(self):
        Z = []
        for i in range(len(landmarks)):
            dist = sqrt((self.x - landmarks[i][0]) ** 2 + (self.y - landmarks[i][1]) ** 2)
            dist += random.gauss(0.0, self.sense_noise)
            Z.append(dist)
        return Z
    
    
    def move(self, turn, forward):
        if forward < 0:
            raise ValueError('Robot cant move backwards')
        
        # turn, and add randomness to the turning command
        orientation = self.orientation + float(turn) + random.gauss(0.0, self.turn_noise)
        orientation %= 2 * pi
        
        # move, and add randomness to the motion command
        dist = float(forward) + random.gauss(0.0, self.forward_noise)
        x = self.x + (cos(orientation) * dist)
        y = self.y + (sin(orientation) * dist)
        x %= world_size    # cyclic truncate
        y %= world_size
        
        # set particle
        res = robot()
        res.set(x, y, orientation)
        res.set_noise(self.forward_noise, self.turn_noise, self.sense_noise)
        return res
    
    def Gaussian(self, mu, sigma, x):
        
        # calculates the probability of x for 1-dim Gaussian with mean mu and var. sigma
        return exp(- ((mu - x) ** 2) / (sigma ** 2) / 2.0) / sqrt(2.0 * pi * (sigma ** 2))
    
    
    def measurement_prob(self, measurement):
        
        # calculates how likely a measurement should be
        
        prob = 1.0;
        for i in range(len(landmarks)):
            dist = sqrt((self.x - landmarks[i][0]) ** 2 + (self.y - landmarks[i][1]) ** 2)
            prob *= self.Gaussian(dist, self.sense_noise, measurement[i])
        return prob
    
    
    
    def __repr__(self):
        return '[x=%.6s y=%.6s orient=%.6s]' % (str(self.x), str(self.y), str(self.orientation))



def eval(r, p):
    sum = 0.0;
    for i in range(len(p)): # calculate mean error
        dx = (p[i].x - r.x + (world_size/2.0)) % world_size - (world_size/2.0)
        dy = (p[i].y - r.y + (world_size/2.0)) % world_size - (world_size/2.0)
        err = sqrt(dx * dx + dy * dy)
        sum += err
    return sum / float(len(p))

#### Quiz
Once again, your robot starts at 30, 50, heading north (pi/2), then turns clockwise by pi/2, moves 15 meters, senses, then turns clockwise by pi/2 again, moves 10 m, then senses again.

In [10]:


#
# Once again, your robot starts at 30, 50,
# heading north (pi/2), then turns clockwise
# by pi/2, moves 15 meters, senses,
# then turns clockwise by pi/2 again, moves
# 10 m, then senses again.
#
# Your program should print out the result of
# your two sense measurements.
#
# Don't modify the code below. Please enter
# your code at the bottom.

####   DON'T MODIFY ANYTHING ABOVE HERE! ENTER CODE BELOW ####

myrobot = robot()
# enter code here
myrobot.set(30.0, 50.0, pi/2)
myrobot = myrobot.move(-pi/2, 15.0)
print(myrobot.sense())
myrobot = myrobot.move(-pi/2, 10.0)
print(myrobot.sense())


[39.05124837953327, 46.09772228646444, 39.05124837953327, 46.09772228646444]
[32.01562118716424, 53.150729063673246, 47.16990566028302, 40.311288741492746]


#### Quiz - Add noise

In [12]:
# Now add noise to your robot as follows:
# forward_noise = 5.0, turn_noise = 0.1,
# sense_noise = 5.0.

myrobot = robot()
# enter code here
myrobot.set_noise(5.0,0.1,5.0)
myrobot.set(30.0, 50.0, pi/2)
myrobot = myrobot.move(-pi/2, 15.0)
print(myrobot.sense())
myrobot = myrobot.move(-pi/2, 10.0)
print(myrobot.sense())

[29.140474452882145, 45.67318479057699, 32.04842999656922, 53.787046787267585]
[36.3212021822161, 60.067146971149455, 50.431726352492475, 35.44666239083387]


### Robot World
Now we know about our class robotwho can turn and then move straight after the turn,and which it also can sense the distance to 4 designated landmarks,L1, L2, L3, and L4, and these distancescomprise the measurement vector of the robot.We told you the robot lives in a world of size 100 x 100,and what this means if this robot falls off 1 end,it disappears on the other.So, it's a cyclic world.So, let's now talk particle filters.

#### Quiz - Creating Particles
The particle filter that you're going to program maintains a set of 1000 random guesses as to where the reward might be. Now, I'm not going to draw 1000 dots here,but let me explain how each of these dots looks like.
Suppose particle is at [38.2, 12.4, 0.18]
Each of these dots is a vector which contains an X coordinate,in this case 38.2, a Y coordinate 12.4,and a heading direction of 0.18,which is the angle at which there are points relative to the X axis. So, this one moves forward, it will move slightly upwards.

In fact, now a code--every time you call the function robotand assign it say to a particle,here the [i] particle,these elements p[i]x, y, and orientation,which is the same as heading,are initialized at random. So, to make a particle set of 1000 particleswhat you have to program is a simple piece of code that assigns 1000 of those to a list.So, let's do this; let me set N=1000 for 1000 particles.Here's my initial set of particles; it's going to be an empty list,and I want you to fill in some codeafter which we have 1000 particles assigned to this vector over here.So, when I print the length of this thingI will get 1000 instead of 0.

In [13]:
# Now we want to create particles,
# p[i] = robot(). In this assignment, write
# code that will assign 1000 such particles
# to a list.
#
# Your program should print out the length
# of your list (don't cheat by making an
# arbitrary list of 1000 elements!)
#
# Don't modify the code below. Please enter
# your code at the bottom.

####   DON'T MODIFY ANYTHING ABOVE HERE! ENTER CODE BELOW ####

N = 1000
p = []

#enter code here
for i in range(N):
    x = robot()
    p.append(x)

print (len(p))

1000


#### Quiz - Moving robot particles
I now want you to take each of these particlesand simulate robot motion.Depending on the heading direction,this might yield a different direction.So, each of these particles shall first turn by 0.1 and then move by 5 meters.We already implemented something just like this for individual robot motion.Now I'd like you to apply this to the entire particle set.So, please go back to the code and make a new set Pthat is the result of this specific motion turning by 0.1and moving forward by 5.0to all of those particles in P.

In [15]:
# Now we want to simulate robot
# motion with our particles.
# Each particle should turn by 0.1
# and then move by 5. 
####   DON'T MODIFY ANYTHING ABOVE HERE! ENTER CODE BELOW ####

N = 1000
p = []
for i in range(N):
    x = robot()
    p.append(x)

p2 = []
for i in range(N):
    p2.append(p[i].move(0.1, 5.0))
print(p) #PLEASE LEAVE THIS HERE FOR GRADING PURPOSES


[[x=24.529 y=93.113 orient=4.0192], [x=40.392 y=68.722 orient=5.2384], [x=50.269 y=40.247 orient=6.1819], [x=58.781 y=63.544 orient=4.1885], [x=8.4685 y=48.226 orient=3.2641], [x=36.518 y=79.215 orient=2.7342], [x=60.883 y=65.555 orient=0.0264], [x=50.743 y=25.668 orient=0.9092], [x=70.806 y=60.322 orient=5.7486], [x=90.071 y=14.260 orient=1.5406], [x=49.557 y=27.466 orient=1.1992], [x=63.730 y=79.247 orient=3.5118], [x=74.218 y=21.314 orient=1.8266], [x=52.877 y=44.651 orient=5.8445], [x=18.925 y=96.651 orient=5.1196], [x=39.736 y=0.8592 orient=2.1066], [x=96.702 y=62.110 orient=3.7470], [x=67.705 y=40.808 orient=0.5588], [x=19.718 y=38.191 orient=2.2172], [x=49.032 y=44.183 orient=4.8440], [x=18.827 y=1.1396 orient=0.7202], [x=98.262 y=6.4692 orient=3.6464], [x=33.405 y=22.100 orient=1.6569], [x=75.610 y=70.667 orient=1.4956], [x=77.224 y=85.090 orient=3.8624], [x=22.665 y=19.818 orient=0.2513], [x=63.511 y=87.232 orient=5.2877], [x=77.648 y=0.2053 orient=4.1557], [x=4.8315 y=14.338 

### Quiz - Importance Weight
Let me explain how the second half works.Suppose an actual robot sits over here,and it measures these exact distances to the landmarks over here.Obviously, there some measurement noise thatwould be just more or less an added Gaussian with 0 mean.Meaning there would be a certain chance of being too short or too longand that probability is governed by a Gaussian.So, this process gives us a measurement vector of 4 valuesof those 4 distances to the landmarks L1 to L4.

Now let's consider a particle that hypothesizes the robot coordinates are over here and not over here, and it also hypothesizes a different heading direction. We can then take the measurement vector and apply it to this particle.Obviously this would be a very poor measurement vectorfor this specific particle over here.In particular, the measurement vector we would've expected looks more like this.That just makes this specific location really unlikely.In fact, the closer our particle to the correct positionthe more likely will be the set of measurements given that position.

And now here comes the big trick in particle filters:the mismatch of the actual measurement and the predicted measurementleads to a so called importance weightthat tells us how important that specific particle is.The larger the weight the more important it is. Well, we now have many, many different particles and a specific measurement.Each of these particles will have a different weight.Some look very plausible, others might look very implausibleas indicated by the size of the circles over here. We now let these particles survive somewhat at random,but the probability of survival will be proportional to their weights.If something has a very big weight like this guy over herewill survive at a higher proportion thansomeone with a really small weight over here, which means after what's called resampling,which is just a technical term for randomly drawing N new particles from these old ones with replacement in proportion to the importance weight.After that resampling phase,those guys over here very likely to live on, in fact many, many times. 

Whereas those guys over here likely have died out.That's exactly what happened in our movie in the beginning when we looked at localization in this corridor environment. The particles that are very consistent with the sensor measurement survive with a higher probability, and the ones with lower importance weight tended to die out.So, we get the fact that the particles clusteraround regions of higher posterior probability.That is really cool and all we have to do iswe have to implement a method for setting importance weightsand that is, of course, related to the likelihood of a measurement,as we will find out, and we have to implement a method for resamplingthat grabs particles in proportion to those weights.
So, let's just do this.So, let me add back the robot code.We built a robot, and we make the robot move,and we now get a sensor measurement for that specific robot using the sense function. So, let's just print this out. These are the ranges or distances to the 4 landmarksand by adding your print my robot statementyou can also figure out weight importance as 33, 48, 0.5, obviously this is a random output because you randomly initialized the position of the robot.

What I want you to program now is a way to assign importance weights to each of the particles in here.I want you to make a list of 1000 elements where each element on the list contains a number.So, this number is proportional to how important that particle is,and to make things easier I coded for you a function in the class robotcalled the measurement probability. This function accepts a single parameter, the measurement vector,the Z edge as defined, and it calculates as an output how likely this measurement is, and it uses effectively a Gaussian that measures how far away the predicted measurements would be from the actual measurements.You can dive into this code and understand what's going on.There's one last change we have to do to make this code run.We have to actually assume that there is measurement noise.If there is 0 measurement noise, then this function will end up dividing by 0.So, let's put in a clause that specifies what we believe the actual measurement noise is. I'm going to do this not for the robot,but I do this for the particles.In this line of code over here where we create the particles for the first time,I now just initialized these positions remain numbersbut also assume a certain amount of noise that goes with each particle,and 5.0 for the measurement noise in those ranges.So, this is the crucial number over here. So, coming back to what I want you to do,I wish you to construct a list of 1000 elements in Wso that each number in this vector reflects the output of thefunction measurement probe applied to the measurement Z that we receive fromthe rear robot, such that when I hit print W,I actually get a list of 1000 importance weights.

In [ ]:
myrobot = robot()
myrobot = myrobot.move(0.1, 5.0)
Z = myrobot.sense()

N = 1000
p = []
for i in range(N):
    x = robot()
    x.set_noise(0.05, 0.05, 5.0)
    p.append(x)

p2 = []
for i in range(N):
    p2.append(p[i].move(0.1, 5.0))
p = p2

w = []
for i in range(N):
    w.append(p[i].measurement_prob(Z))
#insert code here!
print w #Please print w for grading purposes.

### Resampling
* Particle filter here maintains n=1000 random guesses as to where we are. Implement using 1000 robots.
* Importance weight: Mismatch of actual and predicted measurement.
    * The larger the weight, the more important the measurement.
    * The higher the importance (weight), the higher the probability of the particle 'surviving'
    * Usually assumes there is measurement noise.
* Resampling: Randomly redrawing particles from existing ones with replacement (probability of drawing proportional to weight)
* Particles cluster around regions of high posterior probability.

![](images/13.5.png)

#### Replacement
![](images/13.6a.png)

Lets have a small quiz:
We have 5 particles so, N = 5. Now They have following weight:
w = [0.6, 1.2, 2.4, 0.6, 1.2]

If I, in the process of resampling randomly draw a particle in accordance to the normalized importance weights.
What is the probability of drawing P1, P2, P4 and P5?

Answer:
Total Weight = sum(w) = 6 
normalized weight = [0.1, 0.2, 0.4, 0.1, 0.2]

And normalized weight is the answer.

##### Quiz - Is it possible that p1 be never sampled in the resampling step
Answer: yes
And the answer is yes, in the random resampling process something with an importance weight of 0.1 is actually quite unlikely to be sampled into the next data set.

##### Quiz - Is it possible that p3(instead of p1) be never sampled in the resampling step
Answer: yes
And the answer is yes, again. Even though this importance weight over here is large, it could happen that in each of the 5 resampling steps we pick one of the other 4.


##### Quiz - What is the probability of never sampling p3?
** Answer **: 0.0777 
** Solution **
And the answer is 0.0777 approximately, and the way to obtain this is for this particle to never to be drawn in the resampling phase, we always have to draw 1 of the following 4 particles. Those together have a probability of 0.6 to be drawn, which contrasts to the 0.4 for P3. 

So for 5 independent samplings to draw 1 of those 4, we get a total probability of 0.6 to the fifth, which is approximately 0.0777. Put differently, there is about a 7.7% chance that this particle over here is missing, but with almost 93% probability we'd have this particle included. If we hadn't set up P3, gone for P1 over here, which has a much smaller probability of being drawn, then this 0.07 will be as large as 0.59, which is 0.9 to the fifth. Now this means with about 60% chance we will lose particle 1, and only with a 40% chance it will include it. Put differently, the particles with small importance weights will survive at a much lower rate than the ones with larger importance weights, which is exactly what we wish to get from the resampling step.

### Quiz - New Particle
What I would like you to do next is to modify our algorithm to take the lists of particles and importance weights to sample N times the replacement and new particles with a sampling probability proportional to the importance weights, or in the code now that we calculated our new particles and the corresponding importance weights, construct a new particle set P3, which we will call P, again, when everything is said and done, so that the particles in P3 are drawn from P according to the importance weight's W. 

Now to warn you this is more difficult than it looks like. I'm going to show you a trick in a second, so if you fail to do this don't worry. I give you a chance to do it right now, but then I'm going to tell you a little bit more about how to organize this in an efficient way and you get a second chance. So, try it out, see if you can do it, and if you fail look for my advice and then try it again. 

### Resampling Wheel


![](images/13.6b.png)
![](images/13.6c.png)

#### Implementing resampling: Resampling wheel


```
while w[index] < beta:
    beta = beta - w[index]
    index = index + 1

select p[index]
```

Particle is picked in proportion to the proportion of the circumference its slice occupies.


In [ ]:
# In this exercise, you should implement the
# resampler shown in the previous video.

####   DON'T MODIFY ANYTHING ABOVE HERE! ENTER CODE BELOW ####
myrobot = robot()
myrobot = myrobot.move(0.1, 5.0)
Z = myrobot.sense()

N = 1000
p = []
for i in range(N):
    x = robot()
    x.set_noise(0.05, 0.05, 5.0)
    p.append(x)

p2 = []
for i in range(N):
    p2.append(p[i].move(0.1, 5.0))
p = p2

w = []
for i in range(N):
    w.append(p[i].measurement_prob(Z))

p3 = []
max_weight = max(w)
beta = 0.0
# or index = int(random.random() * N)
index = random.randint(0, N-1)
for i in range(N):
    beta += random.random()*(2.0 * max_weight)
    if beta > w[index]:
        beta -= w[index]
        index = (index + 1) % N
    p3.append(p[index])
    
p = p3
print p #please leave this print statement here for grading!



Result: co-located particles. Orientations are not similar because only one location was taken.

### Orientation
* Orientation eventually matters

Now the quiz2 - 

In [ ]:
# Please only modify the indicated area below!


####   DON'T MODIFY ANYTHING ABOVE HERE! ENTER/MODIFY CODE BELOW ####
myrobot = robot()
myrobot = myrobot.move(0.1, 5.0)
Z = myrobot.sense()
N = 1000
T = 10 #Leave this as 10 for grading purposes.

p = []
for i in range(N):
    r = robot()
    r.set_noise(0.05, 0.05, 5.0)
    p.append(r)

for t in range(T):
    myrobot = myrobot.move(0.1, 5.0)
    Z = myrobot.sense()

    p2 = []
    for i in range(N):
        p2.append(p[i].move(0.1, 5.0))
    p = p2

    w = []
    for i in range(N):
        w.append(p[i].measurement_prob(Z))

    p3 = []
    index = int(random.random() * N)
    beta = 0.0
    mw = max(w)
    for i in range(N):
        beta += random.random() * 2.0 * mw
        while beta > w[index]:
            beta -= w[index]
            index = (index + 1) % N
        p3.append(p[index])
    p = p3
    print(eval(myrobot, p))
print(p)

#### Quiz - Error
Just add print eval(myrobot, p) to for loop above.


### Formalisation

![](images/13.7.png)

### Differences between lesson implementation and Google Car implementation
* Robot model: uses Bicycle Model
* Sensor data: Matching snapshots to maps (vs using landmarks)

![](images/13.8.png)

#### Preview
Next: How to use our info to decide where to move. Applying search to the problem of planning robot trajectories in real time.